In [87]:
import pandas as pd
import numpy as np

In [88]:
data = pd.read_csv("pizza_data.csv")
data

,Pizza_ID,Category,Pepperoni,Chicken,Spinach,Pineapple,6_inch_pizza,12_inch_pizza,18_inch_pizza,Nikhil_eats,Charlie_eats
0,1,Training,1,1,1,1,1,0,0,0,0
1,2,Training,0,0,0,1,1,0,0,1,0
2,3,Training,0,0,1,0,1,0,0,0,0
3,4,Training,0,1,0,1,1,0,0,1,0
4,5,Training,1,1,0,0,1,0,0,0,0
5,6,Training,1,1,1,0,0,1,0,0,0
6,7,Training,0,1,0,0,0,1,0,1,0
7,8,Training,0,0,1,1,0,1,0,0,0
8,9,Training,0,0,1,0,0,1,0,1,1
9,10,Training,0,0,0,1,0,1,0,1,0


In [124]:
# drop 18_inch_pizza to avoid dummy trap
X_train, X_test = data.iloc[0:20,2:8], data.iloc[20:25,2:8]

nikhil_train, nikhil_test = data.iloc[0:20, 9:10], data.iloc[20:25, 9:10]
charlie_train, charlie_test = data.iloc[0:20, 10:11], data.iloc[20:25, 10:11]
X_train

,Pepperoni,Chicken,Spinach,Pineapple,6_inch_pizza,12_inch_pizza
0,1,1,1,1,1,0
1,0,0,0,1,1,0
2,0,0,1,0,1,0
3,0,1,0,1,1,0
4,1,1,0,0,1,0
5,1,1,1,0,0,1
6,0,1,0,0,0,1
7,0,0,1,1,0,1
8,0,0,1,0,0,1
9,0,0,0,1,0,1


## Predicting Nikhil's pizza preferences

In [90]:
import statsmodels.api as sm

n_logit = sm.Logit(nikhil_train, X_train)

n_result = n_logit.fit()

print(n_result.summary())

         Current function value: 0.210678
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:            Nikhil_eats   No. Observations:                   20
Model:                          Logit   Df Residuals:                       14
Method:                           MLE   Df Model:                            5
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                  0.6870
Time:                        21:26:22   Log-Likelihood:                -4.2136
converged:                      False   LL-Null:                       -13.460
                                        LLR p-value:                  0.002388
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Pepperoni       -49.0693    5.6e+05  -8.76e-05      1.000    -1.1e+06     1.1e+06
Chicken          19.7430   1.46e+04      0.001      0.99

/usr/local/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [111]:
print(np.exp(n_result.params).round(1))


Pepperoni                0.0
Chicken          375223202.0
Spinach                  0.3
Pineapple                0.4
6_inch_pizza             3.1
12_inch_pizza           11.0
dtype: float64


If chicken appears on the pizza (and there is no pepperoni), all else being equal, the odds that Nikhil eats the slice multiply by 375223202.0. However, if there is a little bit of pepperoni, then it's no deal; odds will go to basically 0. 

In [92]:
n_predicted = n_result.predict(X_test)
n_predicted = np.array([0 if y < 0.5 else 1 for y in n_predicted])

# convert nikhil_test from df to just 1x5 numpy array
nikhil_test = nikhil_test.values.ravel()

n_matches = n_predicted == nikhil_test

## Predicting Charlie's pizza preferences

In [119]:
c_logit = sm.Logit(charlie_train, X_train)

c_result = c_logit.fit()

print(c_result.summary())

         Current function value: 0.187403
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:           Charlie_eats   No. Observations:                   20
Model:                          Logit   Df Residuals:                       14
Method:                           MLE   Df Model:                            5
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                  0.7215
Time:                        21:48:05   Log-Likelihood:                -3.7481
converged:                      False   LL-Null:                       -13.460
                                        LLR p-value:                  0.001602
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Pepperoni        33.2551   8.39e+06   3.96e-06      1.000   -1.64e+07    1.64e+07
Chicken         -64.7055   9.34e+06  -6.93e-06      1.00

/usr/local/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [123]:
print(np.exp(c_result.params).round(2))

Pepperoni        2.770169e+14
Chicken          7.920483e-29
Spinach          1.961992e+14
Pineapple        5.096861e-15
6_inch_pizza     9.674432e-32
12_inch_pizza    1.000000e+00
dtype: float64


From the results of our regression, it looks like Charlie hates small pizzas. He seems to only eats medium and large slices. Charlie also seems to be allergic to chicken and pineapple

In [125]:
c_predicted = c_result.predict(X_test)
c_predicted = np.array([0 if y < 0.5 else 1 for y in c_predicted])

# convert charlie_test from df to just 1x5 numpy array
charlie_test = charlie_test.values.ravel()

c_matches = c_predicted == charlie_test
c_matches

array([False,  True,  True, False, False], dtype=bool)

In [132]:
total_matches = np.concatenate([c_matches, n_matches])
hits = total_matches.sum()
total = len(total_matches)
acc = hits/total*100.0
print("Accuracy: " + str(acc) + "%")

Accuracy: 60.0%
